In [1]:
# US only

import pandas as pd
import numpy as np
import datetime
import pandas as pd
from googleads import adwords

import sys
sys.path.append('/home/jian/Projects/Saatva/To_Lexie_and_Connor/')

import google_analytics_reporting_api as ga
import os

os.getcwd()

'/home/jian/Projects/Saatva/Analysis/President_Day_2019'

In [2]:
def convert_to_date(x):
    try:
        y=datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").date()
    except:
        try:
            y=datetime.datetime.strptime(x,"%m/%d/%Y").date()
        except:
            try:
                y=datetime.datetime.strptime(x,"%B%d/%Y").date()
            except:
                try:
                    y=datetime.datetime.strptime(x,"%b%d/%y").date()
                except:
                    try:
                        y=datetime.datetime.strptime(x,"%m/%d/%y").date()
                    except:
                        y=np.nan
    return y

In [3]:
def convert_to_time(input_x):

    if input_x[len(input_x)-1].lower()=="p":
        x=input_x[:len(input_x)-1]+" PM"
    elif input_x[len(input_x)-1].lower()=="a":
        x=input_x[:len(input_x)-1]+" AM"
    else:
        x=input_x
        
    
    if ((len(x.split(" ")[0].split("-")[0])==4) & (len(x.split(" "))>=2) & ("-" in x)):
        x_0=x.replace(x.split(" ")[0]+" ","").replace(x.split(" ")[0],"")

    else:
        x_0=x
        
        
    if "AM" in x_0:
        x_1=x_0.replace(" AM","").replace("AM","")
        x_list=x_1.split(":")
        if len(x_list)==2:
            x_list=x_list+["00"]
            if x_list[0]=="12":
                x_list[0]="00"
            
            x_1=":".join(x_list)
            
            
        else:
            if x_list[0]=="12":
                x_list[0]="00"
            x_1=":".join(x_list)
        
        
    elif "PM" in x_0:
        x_1=x_0.replace(" PM","").replace("PM","")
        x_list=x_1.split(":")
        if len(x_list)==2:
            x_list=x_list+["00"]
            if x_list[0]!="12":
                x_list[0]=str(int(x_list[0])+12)     
                
            x_1=":".join(x_list)

        else:
            if x_list[0]!="12":
                x_list[0]=str(int(x_list[0])+12)            
            x_1=":".join(x_list)
    else:
        x_1=x_0
    try:
        y=datetime.datetime.strptime(x_1,"%H:%M:%S").time()
    except:
        y=np.nan
        print("Final x_1",x_1)
        print("Final Error",x)

    return y

In [4]:
TV_logs_1=pd.ExcelFile("/home/jian/Projects/Saatva/Analysis/President_Day_2019/TV_logs/Saatva Post Log File_2.11 - 2.17 R1.xlsx")
print(TV_logs_1.sheet_names)
TV_logs_1=TV_logs_1.parse("Sheet1",skiprows=1,dtype=str)

['Sheet1', 'Sheet2']


In [5]:
TV_logs_2=pd.ExcelFile("/home/jian/Projects/Saatva/Analysis/President_Day_2019/TV_logs/Saatva Post Log File_2.18-2.24.xlsx")
print(TV_logs_2.sheet_names)
TV_logs_2=TV_logs_2.parse("Sheet1",skiprows=1,dtype=str)

['Sheet1', 'Sheet2']


In [6]:
TV_logs_1['log_date']=TV_logs_1['Post Log Date'].apply(lambda x: convert_to_date(x))

TV_logs_2['log_date']=TV_logs_2['Post Log Date'].replace('2/24/19 0','2/24/19')
TV_logs_2['log_date']=TV_logs_2['log_date'].apply(lambda x: convert_to_date(x))

In [7]:
TV_logs_1['log_time']=TV_logs_1['Post Log Time'].apply(lambda x: x.replace("XM","AM"))
TV_logs_2['log_time']=TV_logs_2['Post Log Time'].apply(lambda x: x.replace("XM","AM"))

TV_logs_1['log_time']=TV_logs_1['log_time'].apply(lambda x: convert_to_time(x))
TV_logs_2['log_time']=TV_logs_2['log_time'].apply(lambda x: convert_to_time(x))



In [8]:
President_Day_TV_logs=TV_logs_1.append(TV_logs_2)
President_Day_TV_logs['Market']=President_Day_TV_logs['Market'].apply(lambda x: x.strip())




In [9]:
President_Day_TV_logs['date_time_local']=President_Day_TV_logs['log_date'].astype(str)+" "+President_Day_TV_logs['log_time'].astype(str)
President_Day_TV_logs['date_time_local']=President_Day_TV_logs['date_time_local'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))


dict_mkt_time_adj={}
dict_mkt_time_adj.update({"Charlotte":0})
dict_mkt_time_adj.update({"Denver":2})
dict_mkt_time_adj.update({"San Francisco":3})
dict_mkt_time_adj.update({"Chicago":1})
dict_mkt_time_adj.update({"New York":0})
dict_mkt_time_adj.update({"Minneapolis":1})
dict_mkt_time_adj.update({"Washington DC":0})

President_Day_TV_logs=President_Day_TV_logs.reset_index()
del President_Day_TV_logs['index']

President_Day_TV_logs['date_time_NYT']=np.nan
for i in range(len(President_Day_TV_logs)):
    mkt=President_Day_TV_logs['Market'][i]
    date_time_local=President_Day_TV_logs['date_time_local'][i]
    time_adj=dict_mkt_time_adj[mkt]
    date_time_NYT=date_time_local+datetime.timedelta(hours=time_adj)
    President_Day_TV_logs['date_time_NYT'][i]=date_time_NYT
    

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
President_Day_TV_logs['Date_NYT']=President_Day_TV_logs['date_time_NYT'].apply(lambda x: x.date())
President_Day_TV_logs['Hour_NYT']=President_Day_TV_logs['date_time_NYT'].apply(lambda x: int(str(x).split(" ")[1].split(":")[0]))
President_Day_TV_logs=President_Day_TV_logs[(President_Day_TV_logs['date_time_local']>=datetime.date(2019,2,11)) & (President_Day_TV_logs['date_time_local']<datetime.date(2019,2,25))]

# GA_Sessions

In [11]:
# GA New York Time Zone

analytics = ga.initialize_analyticsreporting('/home/jian/Projects/Saatva/GA/GA_api_jubapluscc@gmail.com.json')

def num_func_trans(df):
    if 'ga:sessions' in df.columns:
        df['ga:sessions']=df['ga:sessions'].apply(lambda x: int(x.replace(",","")))
        df['ga:sessions']=df['ga:sessions'].astype(int)

    if 'ga:adClicks' in df.columns:
        df['ga:adClicks']=df['ga:adClicks'].apply(lambda x: int(x.replace(",","")))
        df['ga:adClicks']=df['ga:adClicks'].astype(int)

    if 'ga:impressions' in df.columns:
        df['ga:impressions']=df['ga:impressions'].apply(lambda x: int(x.replace(",","")))
        df['ga:impressions']=df['ga:impressions'].astype(int)
    
    if 'ga:transactions' in df.columns:
        df['ga:transactions']=df['ga:transactions'].apply(lambda x: int(x.replace(",","")))
        df['ga:transactions']=df['ga:transactions'].astype(int)
    
    if 'ga:transactionRevenue' in df.columns:
        df['ga:transactionRevenue']=df['ga:transactionRevenue'].apply(lambda x: float(x.replace(",","")))
        df['ga:transactionRevenue']=df['ga:transactionRevenue'].astype(float)

    
    return df


# BY DAY WITHOUT HOUR

In [12]:
session_by_date_start="2019-02-11"
session_by_date_end="2019-02-24"

In [13]:
ga_data_saatva=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=session_by_date_start, end_date=session_by_date_end,
                                            ga_metrics=['ga:sessions'],
                                            ga_dimensions=['ga:date','ga:sourceMedium','ga:country','ga:metro'])
ga_data_saatva=ga_data_saatva[ga_data_saatva['ga:country']=="United States"]
ga_data_saatva['View']="Saatva.com"
ga_data_saatva['View_ID']="122377905"
print(ga_data_saatva.shape)
#########
ga_data_SM=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=session_by_date_start, end_date=session_by_date_end,
                                            ga_metrics=['ga:sessions'],
                                            ga_dimensions=['ga:date','ga:sourceMedium','ga:country','ga:metro'])

ga_data_SM=ga_data_SM[ga_data_SM['ga:country']=="United States"]
ga_data_SM['View']="Saatvamattress.com"
ga_data_SM['View_ID']="33605697"
print(ga_data_SM.shape)

(6132, 7)
(38640, 7)


In [14]:
ga_data=ga_data_SM.append(ga_data_saatva)
ga_data=num_func_trans(ga_data)

ga_data['Source']=ga_data['ga:sourceMedium'].apply(lambda x: x.split(" / ")[0])
ga_data['Medium']=ga_data['ga:sourceMedium'].apply(lambda x: x.split(" / ")[1])

ga_data['Defined_Media']=np.nan
ga_data['Defined_Media']=np.where(((ga_data['ga:sourceMedium'].str.contains('youtube.')) | (ga_data['Medium']=="social")),"Social",
                                  np.where(((ga_data['Source'].isin(['msn','instagram','Instagram','pinterest','facebook'])) & (ga_data['Medium']=='cpc')),"Social",
                                           np.where(ga_data['ga:sourceMedium'].str.contains('criteo'),"Retarget",
                                                    np.where(ga_data['Source']=="(direct)","Direct",
                                                             np.where(((ga_data['Source'].isin(['google','bing','yahoo'])) & (ga_data['Medium']=='cpc')),"SEM",
                                                                      np.where(((ga_data['Source'].isin(['google','bing','yahoo'])) & (ga_data['Medium']=='organic')),"SEO",
                                                                               np.where(ga_data['Source']=="powerInbox","Email",
                                                                                        np.where((ga_data['ga:sourceMedium'].str.contains('email')) & (~ga_data['ga:sourceMedium'].isin(["taboola / townnews-wvgazettemail","query.hemailaccessonline.com / referral"]) | (ga_data['ga:sourceMedium'].str.contains("mail.") )),"Email",
                                                                                                 np.where(ga_data['ga:sourceMedium'].str.contains('taboola'),"Video",
                                                                                                          np.where(ga_data['Medium']=="referral","Referral",
                                                                                                                   np.where(ga_data['Medium'].isin(["partner","partners"]),"partner",
                                                                                                                            np.where(ga_data['ga:sourceMedium'].str.contains('display'),"Display","Others")
                                                                                                                            )
                                                                                                                   )
                                                                                                          )
                                                                                                )
                                                                                       )
                                                                              )
                                                                     )
                                                            )
                                                   )
                                          )
                                 )

In [15]:
GA_taxonomy=ga_data[['ga:sourceMedium','Source','Medium','Defined_Media']].drop_duplicates()
GA_taxonomy.reset_index(inplace=True)
GA_taxonomy=GA_taxonomy.sort_values('Defined_Media')
del GA_taxonomy['index']

In [16]:
ga_agg_session=ga_data.groupby(['ga:date','ga:metro','Defined_Media'])['ga:sessions'].sum().to_frame()
ga_agg_session.reset_index(inplace=True)
ga_agg_session['ga:date']=ga_agg_session['ga:date'].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d").date())

In [17]:
ga_agg_session['Defined_Media'].unique()

array(['Direct', 'Others', 'Referral', 'SEM', 'SEO', 'Social', 'partner',
       'Display', 'Email', 'Retarget', 'Video'], dtype=object)

In [18]:
Market_TV_to_GoogleAdwords={}
Market_TV_to_GoogleAdwords.update({"Charlotte":"Charlotte NC"})
Market_TV_to_GoogleAdwords.update({"Chicago":"Chicago IL"})
Market_TV_to_GoogleAdwords.update({"Denver":"Denver CO"})
Market_TV_to_GoogleAdwords.update({"Minneapolis":"Minneapolis-St. Paul MN"})
Market_TV_to_GoogleAdwords.update({"New York":"New York NY"})
Market_TV_to_GoogleAdwords.update({"San Francisco":"San Francisco-Oakland-San Jose CA"})
Market_TV_to_GoogleAdwords.update({"Washington DC":"Washington DC (Hagerstown MD)"})
Market_TV_to_GoogleAdwords

{'Charlotte': 'Charlotte NC',
 'Chicago': 'Chicago IL',
 'Denver': 'Denver CO',
 'Minneapolis': 'Minneapolis-St. Paul MN',
 'New York': 'New York NY',
 'San Francisco': 'San Francisco-Oakland-San Jose CA',
 'Washington DC': 'Washington DC (Hagerstown MD)'}

In [19]:
df_TV_copy_day_session=President_Day_TV_logs.copy()
df_TV_copy_day_session=df_TV_copy_day_session.groupby(['Market','Date_NYT'])['log_time'].count().to_frame().reset_index()
df_TV_copy_day_session['GoogleMarket']=df_TV_copy_day_session['Market'].apply(lambda x: Market_TV_to_GoogleAdwords[x])
df_TV_copy_day_session=df_TV_copy_day_session[['GoogleMarket','Date_NYT','log_time']].rename(columns={"log_time":"log_count"})
df_TV_copy_day_session

df_1=pd.DataFrame({"GoogleMarket":df_TV_copy_day_session['GoogleMarket'].unique().tolist()})
df_1['index']=1
df_2=pd.DataFrame({"Date_NYT":df_TV_copy_day_session['Date_NYT'].unique().tolist()})
df_2['index']=1
df_1=pd.merge(df_1,df_2,on="index",how="outer")
del df_1['index']

df_TV_copy_day_session=pd.merge(df_TV_copy_day_session,df_1,on=['GoogleMarket','Date_NYT'],how="outer")
df_TV_copy_day_session=df_TV_copy_day_session[df_TV_copy_day_session['Date_NYT']>=datetime.date(2019,2,11)]
df_TV_copy_day_session=df_TV_copy_day_session[df_TV_copy_day_session['Date_NYT']<=datetime.date(2019,2,24)]
df_TV_copy_day_session['log_count']=df_TV_copy_day_session['log_count'].fillna(0)



In [21]:
for session_type,group in ga_agg_session.groupby("Defined_Media"):
    mkt_list=[]
    writer=pd.ExcelWriter("/home/jian/Projects/Saatva/Analysis/President_Day_2019/output/output_sessions_by_day/Saatva_Sessions_Type_of_"+session_type+"_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
    for mkt in df_TV_copy_day_session['GoogleMarket'].tolist():
        df_GA_mkt=group[group['ga:metro']==mkt].rename(columns={"ga:date":"Date_NYT","ga:metro":"GoogleMarket"})
        df_TV=df_TV_copy_day_session[df_TV_copy_day_session['GoogleMarket']==mkt]

        df_mkt_session=pd.merge(df_GA_mkt,df_TV,on=['Date_NYT','GoogleMarket'],how="outer")
        df_mkt_session['session_type']=session_type
        df_mkt_session=df_mkt_session[['GoogleMarket','Date_NYT','session_type','log_count','ga:sessions']]
        mkt_list=mkt_list+[mkt]
        df_mkt_session.to_excel(writer,mkt.split("-")[0],index=False)
    targeted_7_mkt_TV=df_TV_copy_day_session.groupby(['Date_NYT'])['log_count'].sum().to_frame().reset_index()
    targeted_7_mkt_GA=group[group['ga:metro'].isin(mkt_list)].rename(columns={"ga:date":"Date_NYT","ga:metro":"GoogleMarket"})
    targeted_7_mkt_GA=targeted_7_mkt_GA.groupby(['Date_NYT'])['ga:sessions'].sum().to_frame().reset_index()
    df_targeted_7_mkt=pd.merge(targeted_7_mkt_TV,targeted_7_mkt_GA,on=['Date_NYT'],how="outer")
    df_targeted_7_mkt['GoogleMarket']="Targeted_7_Mkts"
    df_targeted_7_mkt['session_type']="All_Sessions"
    df_targeted_7_mkt=df_targeted_7_mkt[df_mkt_session.columns.tolist()]
    
    other_mkts_GA=group[~group['ga:metro'].isin(mkt_list)].rename(columns={"ga:date":"Date_NYT","ga:metro":"GoogleMarket"})
    other_mkts_GA=other_mkts_GA.groupby(['Date_NYT'])['ga:sessions'].sum().to_frame().reset_index()
    other_mkts_GA['GoogleMarket']="Others"
    other_mkts_GA['session_type']="All_Sessions"
    other_mkts_GA=other_mkts_GA[[x for x in df_mkt_session.columns.tolist() if x !="log_count"]]
    
    df_targeted_7_mkt.to_excel(writer,"targeted_7_mkts",index=False)
    other_mkts_GA.to_excel(writer,"other_mkts",index=False)
    writer.save()

In [23]:
writer=pd.ExcelWriter("/home/jian/Projects/Saatva/Analysis/President_Day_2019/output/output_sessions_by_day/Saatva_All_Sessions_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
for mkt in df_TV_copy_day_session['GoogleMarket'].tolist():
    df_GA_mkt=ga_agg_session[ga_agg_session['ga:metro']==mkt].rename(columns={"ga:date":"Date_NYT","ga:metro":"GoogleMarket"})
    df_GA_mkt=df_GA_mkt.groupby(["Date_NYT","GoogleMarket"])['ga:sessions'].sum().to_frame().reset_index()
    df_TV=df_TV_copy_day_session[df_TV_copy_day_session['GoogleMarket']==mkt]

    df_mkt_session=pd.merge(df_GA_mkt,df_TV,on=['Date_NYT','GoogleMarket'],how="outer")
    df_mkt_session['session_type']="All_Sessions"
    df_mkt_session=df_mkt_session[['GoogleMarket','Date_NYT','session_type','log_count','ga:sessions']]
    mkt_list=mkt_list+[mkt]
    df_mkt_session.to_excel(writer,mkt.split("-")[0],index=False)
targeted_7_mkt_TV=df_TV_copy_day_session.groupby(['Date_NYT'])['log_count'].sum().to_frame().reset_index()
targeted_7_mkt_GA=ga_agg_session[ga_agg_session['ga:metro'].isin(mkt_list)].rename(columns={"ga:date":"Date_NYT","ga:metro":"GoogleMarket"})
targeted_7_mkt_GA=targeted_7_mkt_GA.groupby(['Date_NYT'])['ga:sessions'].sum().to_frame().reset_index()
df_targeted_7_mkt=pd.merge(targeted_7_mkt_TV,targeted_7_mkt_GA,on=['Date_NYT'],how="outer")
df_targeted_7_mkt['GoogleMarket']="Targeted_7_Mkts"
df_targeted_7_mkt['session_type']="All_Sessions"
df_targeted_7_mkt=df_targeted_7_mkt[df_mkt_session.columns.tolist()]
    
other_mkts_GA=ga_agg_session[~ga_agg_session['ga:metro'].isin(mkt_list)].rename(columns={"ga:date":"Date_NYT","ga:metro":"GoogleMarket"})
other_mkts_GA=other_mkts_GA.groupby(['Date_NYT'])['ga:sessions'].sum().to_frame().reset_index()
other_mkts_GA['GoogleMarket']="Others"
other_mkts_GA['session_type']="All_Sessions"
other_mkts_GA=other_mkts_GA[[x for x in df_mkt_session.columns.tolist() if x !="log_count"]]
    
df_targeted_7_mkt.to_excel(writer,"targeted_7_mkts",index=False)
other_mkts_GA.to_excel(writer,"other_mkts",index=False)
writer.save()

# BY DAY BY HOUR

In [ ]:
Start_Date="2019-02-11"
End_Date="2019-02-25" # +1 day for the adwords later

In [13]:
# Saatva.com
# dateHour, sourceMedium      sessions, adClicks, impressions
ga_data_saatva=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date, end_date=End_Date,
                                            ga_metrics=['ga:sessions'],
                                            ga_dimensions=['ga:dateHour','ga:sourceMedium','ga:country','ga:metro'])
ga_data_saatva=ga_data_saatva[ga_data_saatva['ga:country']=="United States"]
ga_data_saatva['View']="Saatva.com"
ga_data_saatva['View_ID']="122377905"
ga_data_saatva.shape

(13613, 7)

In [14]:
# Saatvamattress.com
# dateHour, sourceMedium      sessions, adClicks, impressions
ga_data_SM=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date, end_date=End_Date,
                                            ga_metrics=['ga:sessions'],
                                            ga_dimensions=['ga:dateHour','ga:sourceMedium','ga:country','ga:metro'])

ga_data_SM=ga_data_SM[ga_data_SM['ga:country']=="United States"]
ga_data_SM['View']="Saatvamattress.com"
ga_data_SM['View_ID']="33605697"
ga_data_SM.shape

(167310, 7)

In [15]:
ga_data=ga_data_SM.append(ga_data_saatva)
ga_data=num_func_trans(ga_data)

ga_data['Source']=ga_data['ga:sourceMedium'].apply(lambda x: x.split(" / ")[0])
ga_data['Medium']=ga_data['ga:sourceMedium'].apply(lambda x: x.split(" / ")[1])

ga_data['Defined_Media']=np.nan
ga_data['Defined_Media']=np.where(((ga_data['ga:sourceMedium'].str.contains('youtube.')) | (ga_data['Medium']=="social")),"Social",
                                  np.where(((ga_data['Source'].isin(['msn','instagram','Instagram','pinterest','facebook'])) & (ga_data['Medium']=='cpc')),"Social",
                                           np.where(ga_data['ga:sourceMedium'].str.contains('criteo'),"Retarget",
                                                    np.where(ga_data['Source']=="(direct)","Direct",
                                                             np.where(((ga_data['Source'].isin(['google','bing','yahoo'])) & (ga_data['Medium']=='cpc')),"SEM",
                                                                      np.where(((ga_data['Source'].isin(['google','bing','yahoo'])) & (ga_data['Medium']=='organic')),"SEO",
                                                                               np.where(ga_data['Source']=="powerInbox","Email",
                                                                                        np.where((ga_data['ga:sourceMedium'].str.contains('email')) & (~ga_data['ga:sourceMedium'].isin(["taboola / townnews-wvgazettemail","query.hemailaccessonline.com / referral"]) | (ga_data['ga:sourceMedium'].str.contains("mail.") )),"Email",
                                                                                                 np.where(ga_data['ga:sourceMedium'].str.contains('taboola'),"Video",
                                                                                                          np.where(ga_data['Medium']=="referral","Referral",
                                                                                                                   np.where(ga_data['Medium'].isin(["partner","partners"]),"partner",
                                                                                                                            np.where(ga_data['ga:sourceMedium'].str.contains('display'),"Display","Others")
                                                                                                                            )
                                                                                                                   )
                                                                                                          )
                                                                                                )
                                                                                       )
                                                                              )
                                                                     )
                                                            )
                                                   )
                                          )
                                 )

In [16]:
GA_taxonomy=ga_data[['ga:sourceMedium','Source','Medium','Defined_Media']].drop_duplicates()
GA_taxonomy.reset_index(inplace=True)
GA_taxonomy=GA_taxonomy.sort_values('Defined_Media')
del GA_taxonomy['index']

In [17]:
ga_agg_session=ga_data.groupby(['ga:dateHour','ga:metro','Defined_Media'])['ga:sessions'].sum().to_frame()
ga_agg_session.reset_index(inplace=True)
# ga_agg_wide_session=ga_agg_session.pivot('ga:dateHour','Defined_Media','ga:sessions')
# ga_agg_wide_session.reset_index(inplace=True)
# ga_agg_wide_session['time_hour']=ga_agg_wide_session['ga:dateHour'].apply(lambda x:datetime.datetime.strptime(x,"%Y%m%d%H"))
# ga_agg_wide_session=ga_agg_wide_session.fillna(0)


In [18]:
ga_agg_session.head(4)

,ga:dateHour,ga:metro,Defined_Media,ga:sessions
0,2019021100,(not set),Direct,3
1,2019021100,(not set),Others,1
2,2019021100,(not set),Referral,4
3,2019021100,(not set),SEM,6


# AdWords

In [60]:
Start_Date_AdWords="2019-02-01"
End_Date_AdWords="2019-02-25"



In [30]:
def get_client(config_path='googleads_saatva.yaml'):
    return adwords.AdWordsClient.LoadFromStorage(config_path)

def download_report_as_stream(client, report_type, fields, start_date, end_date):
    '''Download Adwords report, then save as csv file to base_dir
    Args:
        client: adwords.AdWordsClient.LoadFromStorage()
        report_type: str eg. 'KEYWORDS_PERFORMANCE_REPORT'
        fields: list eg. ["CampaignId", "CampaignName", ... ]
        start_date: str, eg. '2018-02-11'
        end_date: str, eg.
    Return:
        str, path to csv file
    '''
    report_downloader = client.GetReportDownloader(version='v201809')
    # Create report definition.
    report = {
        'reportName': report_type,
        'dateRangeType': 'CUSTOM_DATE',
        'reportType': report_type,
        'downloadFormat': 'CSV',
        'selector': {
            'fields': fields,
            'dateRange': {
                'min': start_date,
                'max': end_date}
        }
    }

    # save output
    return report_downloader.DownloadReportAsStream(report, 
        skip_report_header=True, skip_column_header=False,
        skip_report_summary=True)



# 1

client_Saatva = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_Saatva = download_report_as_stream(client_Saatva, report_type, fields, Start_Date_AdWords, End_Date_AdWords)
Saatva_Click = pd.read_csv(response_Saatva)
Saatva_Click['Cost']=Saatva_Click['Cost'].apply(lambda x: x/1000000)
Saatva_Click['Channel_Network']=Saatva_Click['Advertising Channel']+"|"+Saatva_Click['Network']+"|"+Saatva_Click['Network (with search partners)']


# 2 Empty

client_Saatva_Branding = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva_Branding.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_Saatva_Branding = download_report_as_stream(client_Saatva_Branding, report_type, fields, Start_Date_AdWords, End_Date_AdWords)
Saatva_Brand_Click = pd.read_csv(response_Saatva_Branding)
Saatva_Brand_Click['Cost']=Saatva_Brand_Click['Cost'].apply(lambda x: x/1000000)
Saatva_Brand_Click['Channel_Network']=Saatva_Brand_Click['Advertising Channel']+"|"+Saatva_Brand_Click['Network']+"|"+Saatva_Brand_Click['Network (with search partners)']
Saatva_Brand_Click['Channel_Network'].unique()


# 3

client_Saatva_Competitors = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva_Competitors.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_Saatva_Competitor = download_report_as_stream(client_Saatva_Competitors, report_type, fields, Start_Date_AdWords, End_Date_AdWords)
Saatva_Competitor_Click = pd.read_csv(response_Saatva_Competitor)
Saatva_Competitor_Click['Cost']=Saatva_Competitor_Click['Cost'].apply(lambda x: x/1000000)
Saatva_Competitor_Click['Channel_Network']=Saatva_Competitor_Click['Advertising Channel']+"|"+Saatva_Competitor_Click['Network']+"|"+Saatva_Competitor_Click['Network (with search partners)']


# 4

client_TheSaatvaCompany = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_TheSaatvaCompany.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_TheSaatvaCompany = download_report_as_stream(client_TheSaatvaCompany, report_type, fields, Start_Date_AdWords, End_Date_AdWords)
TheSaatvaCompany_Click = pd.read_csv(response_TheSaatvaCompany)
TheSaatvaCompany_Click['Cost']=TheSaatvaCompany_Click['Cost'].apply(lambda x: x/1000000)
TheSaatvaCompany_Click['Channel_Network']=TheSaatvaCompany_Click['Advertising Channel']+"|"+TheSaatvaCompany_Click['Network']+"|"+TheSaatvaCompany_Click['Network (with search partners)']



# 5 Empty

client_TheSaatvaCompanyBranding = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_TheSaatvaCompanyBranding.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_TheSaatvaCompanyBranding = download_report_as_stream(client_TheSaatvaCompanyBranding, report_type, fields, Start_Date_AdWords, End_Date_AdWords)
TheSaatvaCompanyBranding_Click = pd.read_csv(response_TheSaatvaCompanyBranding)
TheSaatvaCompanyBranding_Click['Cost']=TheSaatvaCompanyBranding_Click['Cost'].apply(lambda x: x/1000000)
TheSaatvaCompanyBranding_Click['Channel_Network']=TheSaatvaCompanyBranding_Click['Advertising Channel']+"|"+TheSaatvaCompanyBranding_Click['Network']+"|"+TheSaatvaCompanyBranding_Click['Network (with search partners)']


In [31]:
Saatva_Competitor_Click=Saatva_Competitor_Click[Saatva_Competitor_Click['Campaign'].apply(lambda x: x[len(x)-4:len(x)+1])=="[SM]"]
Saatva_Competitor_Click.reset_index(inplace=True)
del Saatva_Competitor_Click['index']

In [32]:
set(Saatva_Click['Channel_Network'].unique().tolist()+Saatva_Brand_Click['Channel_Network'].unique().tolist()+Saatva_Competitor_Click['Channel_Network'].unique().tolist()+    TheSaatvaCompany_Click['Channel_Network'].unique().tolist()+TheSaatvaCompanyBranding_Click['Channel_Network'].unique().tolist())


{'Display|Display Network|Display Network',
 'Search|Search Network|Google search',
 'Search|Search Network|Search partners',
 'Shopping|Search Network|Google search',
 'Shopping|Search Network|Search partners',
 'Video|Display Network|Display Network',
 'Video|YouTube Search|YouTube Search',
 'Video|YouTube Videos|YouTube Videos'}

In [33]:
df_all_adwords=Saatva_Click.append(Saatva_Brand_Click).append(Saatva_Competitor_Click)
df_all_adwords=df_all_adwords.append(TheSaatvaCompany_Click).append(TheSaatvaCompanyBranding_Click)
df_s=df_all_adwords.groupby(['Channel_Network'])[['Impressions','Clicks','Cost']].sum().reset_index()
Video_summary=df_s[df_s['Channel_Network'].apply(lambda x: "Video" in x)]

In [34]:
Saatva_Click['Campaign_Type']=Saatva_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))
Saatva_Brand_Click['Campaign_Type']=Saatva_Brand_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM_Brand").replace("Shopping","PLA"))
if Saatva_Competitor_Click['Campaign'].apply(lambda x: x[:17]).unique().tolist()==["[ADL] [Non-Brand]"]:
    Saatva_Competitor_Click['Campaign_Type']="SEM_Non_Brand" # Because it's all labeled as Non-Brand in Campaign and in Search Only Campaign Type
TheSaatvaCompany_Click['Campaign_Type']=TheSaatvaCompany_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))
TheSaatvaCompanyBranding_Click['Campaign_Type']=TheSaatvaCompanyBranding_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))


Saatva_Click_Search=Saatva_Click[Saatva_Click['Advertising Channel']=='Search']
Saatva_Click_Others=Saatva_Click[Saatva_Click['Advertising Channel']!='Search']

Saatva_Click_Search['Campaign_Type']=np.where((Saatva_Click_Search['Campaign'].str.contains("Branded")) | (Saatva_Click_Search['Campaign'].str.contains(" Brand ")) | (Saatva_Click_Search['Campaign'].str.contains("Co-Brand")) | (Saatva_Click_Search['Campaign'].str.contains("\[Brand\]")),"SEM_Brand",
                                                 np.where((Saatva_Click_Search['Campaign'].str.contains("Non-Brand")) | (Saatva_Click_Search['Campaign'].str.contains("Dynamic Search Ads")),"SEM_Non_Brand","NA"))
Saatva_Click_Search_1=Saatva_Click_Search[Saatva_Click_Search['Campaign_Type']!="NA"]
Saatva_Click_Search_2=Saatva_Click_Search[Saatva_Click_Search['Campaign_Type']=="NA"]
print("Make sure it's 0: "+str(len(Saatva_Click_Search_2))) # Make sure it's 0



TheSaatvaCompany_Click_Search=TheSaatvaCompany_Click[TheSaatvaCompany_Click['Advertising Channel']=='Search']
TheSaatvaCompany_Click_Others=TheSaatvaCompany_Click[TheSaatvaCompany_Click['Advertising Channel']!='Search']

TheSaatvaCompany_Click_Search['Campaign_Type']=np.where((TheSaatvaCompany_Click_Search['Campaign'].str.contains("Branded")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains(" Brand ")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains("Co-Brand")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains("\[Brand\]")),"SEM_Brand",
                                                 np.where((TheSaatvaCompany_Click_Search['Campaign'].str.contains("Non-Brand")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains("Dynamic Search Ads")),"SEM_Non_Brand","NA"))
TheSaatvaCompany_Click_Search_1=TheSaatvaCompany_Click_Search[TheSaatvaCompany_Click_Search['Campaign_Type']!="NA"]
TheSaatvaCompany_Click_Search_2=TheSaatvaCompany_Click_Search[TheSaatvaCompany_Click_Search['Campaign_Type']=="NA"]
print("Make sure it's 0: "+str(len(TheSaatvaCompany_Click_Search_2))) # Make sure it's 0


Make sure it's 0: 0
Make sure it's 0: 0


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [35]:
Saatva_Click=Saatva_Click_Search.append(Saatva_Click_Others)
TheSaatvaCompany_Click=TheSaatvaCompany_Click_Search.append(TheSaatvaCompany_Click_Others)

In [36]:
Saatva_Agg=Saatva_Click.append(Saatva_Brand_Click).append(Saatva_Competitor_Click).append(TheSaatvaCompany_Click).append(TheSaatvaCompanyBranding_Click)
Saatva_Agg['Day']=Saatva_Agg['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Saatva_Agg.reset_index(inplace=True)
del Saatva_Agg['index']
Saatva_Agg['Hour of day']=Saatva_Agg['Hour of day'].astype(int)



In [37]:
account_campaing_name_type=Saatva_Agg[['Account','Campaign','Campaign_Type']].drop_duplicates()
print(account_campaing_name_type.shape)

b=Saatva_Agg[['Account','Campaign']].drop_duplicates()
print(b.shape)

# Use the Account, Campaign combination to define the media
del b

(213, 3)
(213, 2)


In [38]:
Saatva_Agg=Saatva_Agg.groupby(["Campaign_Type","Day","Hour of day"])['Impressions','Clicks', 'Cost'].apply(sum)
Saatva_Agg.reset_index(inplace=True)
Saatva_Agg['CTR']=Saatva_Agg['Clicks']/Saatva_Agg['Impressions']

Saatva_Agg_No_NA=Saatva_Agg[(Saatva_Agg['Campaign_Type']!="NA") & (Saatva_Agg['Campaign_Type']!="Search Network with Display Select")]


In [39]:
a=Saatva_Click.append(Saatva_Brand_Click).append(Saatva_Competitor_Click).append(TheSaatvaCompany_Click).append(TheSaatvaCompanyBranding_Click)
a['Advertising Channel'].unique()

array(['Search', 'Video', 'Shopping', 'Display'], dtype=object)

In [40]:
AdWords_output=Saatva_Agg.copy()
AdWords_output['Date']=AdWords_output['Day'].apply(lambda x: str(x)[0:4]+str(x)[5:7]+str(x)[8:10])
AdWords_output=AdWords_output[['Campaign_Type','Date','Hour of day','Impressions','Clicks','Cost']]
AdWords_output=AdWords_output.rename(columns={"Hour of day":"Hour","Impressions":"IMP"})
AdWords_output=AdWords_output.sort_values(["Campaign_Type","Date","Hour"],ascending=[True,True,True])


# DMA Level

In [41]:
fields = ["AccountDescriptiveName",'CampaignName',"Date","CountryCriteriaId","MetroCriteriaId","Impressions", "Clicks", "Cost"]
report_type = 'GEO_PERFORMANCE_REPORT'
    
def getting_geo_df(client_yaml, report_type, fields, Start_Date_overall, End_Date_overall):
    response_Saatva = download_report_as_stream(client_yaml, report_type, fields, Start_Date_overall, End_Date_overall)
    df = pd.read_csv(response_Saatva)
    return df

In [42]:
Geo_Saatva = getting_geo_df(client_Saatva, report_type, fields, Start_Date_AdWords, End_Date_AdWords)
Geo_Saatva['Cost']=Geo_Saatva['Cost'].apply(lambda x: x/1000000)

Geo_Saatva_Branding = getting_geo_df(client_Saatva_Branding, report_type, fields, Start_Date_AdWords, End_Date_AdWords)
Geo_Saatva_Branding['Cost']=Geo_Saatva_Branding['Cost'].apply(lambda x: x/1000000)

Geo_Saatva_Competitors = getting_geo_df(client_Saatva_Competitors, report_type, fields, Start_Date_AdWords, End_Date_AdWords)
Geo_Saatva_Competitors['Cost']=Geo_Saatva_Competitors['Cost'].apply(lambda x: x/1000000)

Geo_TheSaatvaCompany = getting_geo_df(client_TheSaatvaCompany, report_type, fields, Start_Date_AdWords, End_Date_AdWords)
Geo_TheSaatvaCompany['Cost']=Geo_TheSaatvaCompany['Cost'].apply(lambda x: x/1000000)

Geo_TheSaatvaCompanyBranding = getting_geo_df(client_TheSaatvaCompanyBranding, report_type, fields, Start_Date_AdWords, End_Date_AdWords)
Geo_TheSaatvaCompanyBranding['Cost']=Geo_TheSaatvaCompanyBranding['Cost'].apply(lambda x: x/1000000)

In [43]:
Geo_Saatva_Competitors=Geo_Saatva_Competitors[Geo_Saatva_Competitors['Campaign'].apply(lambda x: x[len(x)-4:len(x)+1])=="[SM]"]
Geo_Saatva_Competitors.reset_index(inplace=True)
del Geo_Saatva_Competitors['index']



In [44]:
Geo_All_5=Geo_Saatva.append(Geo_Saatva_Branding).append(Geo_Saatva_Competitors).append(Geo_TheSaatvaCompany).append(Geo_TheSaatvaCompanyBranding)
Geo_All_5=Geo_All_5[Geo_All_5['Country/Territory']==2840] # US only
Metro_id_list=Geo_All_5['Metro area'].unique().tolist()
Metro_id_list.remove(" --")
Metro_id_list=Metro_id_list+[int(x) for x in Metro_id_list]

In [45]:
# DMA names

DMA_ID_Name=pd.DataFrame()
count_i=0
for client_ in [client_Saatva,client_Saatva_Branding,client_Saatva_Competitors,client_TheSaatvaCompany,client_TheSaatvaCompanyBranding]:

    location_criterion_service = client_.GetService('LocationCriterionService',version='v201809')

    location_id_list = Metro_id_list

    selector = {
      'fields': ['Id', 'LocationName', 'DisplayType', 'CanonicalName',
                 'ParentLocations', 'Reach', 'TargetingStatus'],
      'predicates': [{
          'field': 'Id',
          'operator': 'IN',
          'values': location_id_list
      }]
    }
    response_list = location_criterion_service.get(selector)
    for response in response_list:
        DMA_id=response['location']['id']
        DMA_type=response['location']['displayType']
        DMA_name=response['location']['locationName']

        df=pd.DataFrame({"DMA_id":DMA_id,"DMA_type":DMA_type,"DMA_name":DMA_name},index=[count_i])
        count_i+=1
        DMA_ID_Name=DMA_ID_Name.append(df)
DMA_ID_Name=DMA_ID_Name.drop_duplicates()
DMA_ID_Name['DMA_id']=DMA_ID_Name['DMA_id'].astype(str)
DMA_ID_Name.head(2)

,DMA_id,DMA_name,DMA_type
0,200693,Little Rock-Pine Bluff AR,DMA Region
1,200617,Milwaukee WI,DMA Region


In [46]:
Geo_All_5=pd.merge(Geo_All_5,DMA_ID_Name,left_on="Metro area",right_on="DMA_id",how="left")

Geo_All_5[pd.isnull(Geo_All_5['DMA_name'])]['Metro area'].unique()
Geo_All_5['DMA_name']=Geo_All_5['DMA_name'].fillna(" --")

In [47]:
Geo_All_5=pd.merge(Geo_All_5,account_campaing_name_type,on=['Account','Campaign'],how="left")
Geo_All_5=Geo_All_5.reset_index()
del Geo_All_5['index']
Geo_All_5=Geo_All_5.groupby(['Day','Campaign_Type','DMA_name'])['Impressions','Clicks','Cost'].apply(sum).reset_index()#.sum()#


# Manage with TV logs

In [48]:
TV_Markets=President_Day_TV_logs['Market'].unique().tolist()

Market_TV_to_GoogleAdwords={}
Market_TV_to_GoogleAdwords.update({"Charlotte":"Charlotte NC"})
Market_TV_to_GoogleAdwords.update({"Chicago":"Chicago IL"})
Market_TV_to_GoogleAdwords.update({"Denver":"Denver CO"})
Market_TV_to_GoogleAdwords.update({"Minneapolis":"Minneapolis-St. Paul MN"})
Market_TV_to_GoogleAdwords.update({"New York":"New York NY"})
Market_TV_to_GoogleAdwords.update({"San Francisco":"San Francisco-Oakland-San Jose CA"})
Market_TV_to_GoogleAdwords.update({"Washington DC":"Washington DC (Hagerstown MD)"})

Market_TV_to_GoogleAnalytics={}
Market_TV_to_GoogleAnalytics.update({"Charlotte":"Charlotte NC"})
Market_TV_to_GoogleAnalytics.update({"Chicago":"Chicago IL"})
Market_TV_to_GoogleAnalytics.update({"Denver":"Denver CO"})
Market_TV_to_GoogleAnalytics.update({"Minneapolis":"Minneapolis-St. Paul MN"})
Market_TV_to_GoogleAnalytics.update({"New York":"New York NY"})
Market_TV_to_GoogleAnalytics.update({"San Francisco":"San Francisco-Oakland-San Jose CA"})
Market_TV_to_GoogleAnalytics.update({"Washington DC":"Washington DC (Hagerstown MD)"})




df_TV_Markets=pd.DataFrame({"TV_Market":TV_Markets})
df_TV_Markets['GoogleAdwords_Market']=df_TV_Markets['TV_Market'].apply(lambda x: Market_TV_to_GoogleAdwords[x])
df_TV_Markets['GoogleAnalytics_Market']=df_TV_Markets['TV_Market'].apply(lambda x: Market_TV_to_GoogleAdwords[x])

print(df_TV_Markets['GoogleAdwords_Market'].tolist()==df_TV_Markets['GoogleAnalytics_Market'].tolist())

df_TV_Markets_rename=df_TV_Markets[['TV_Market',"GoogleAdwords_Market"]].rename(columns={"TV_Market":"Market","GoogleAdwords_Market":"GoogleMarket"}) #GA, Ads the same name

True


In [49]:
df_TV_Market=President_Day_TV_logs.groupby(['Date_NYT','Hour_NYT','Market'])['Post Log Time'].count().to_frame().reset_index().rename(columns={"Post Log Time":"log_count"})
df_TV_Market=df_TV_Market.sort_values(['Date_NYT','Hour_NYT','Market'])
df_TV_Market=df_TV_Market[df_TV_Market['Date_NYT']>=datetime.date(2019,2,11)]
df_TV_Market=pd.merge(df_TV_Markets_rename,df_TV_Market,on="Market",how="left")

df_TV_Market.shape

(695, 5)

In [50]:
df_date=pd.DataFrame({"Date_NYT":[datetime.date(2019,2,11)+datetime.timedelta(x) for x in range(15)],'ind':1})
df_hour=pd.DataFrame({"Hour_NYT":[x for x in range(24)],'ind':1})
df_mkt=pd.DataFrame({"Market":[x for x in df_TV_Market['Market'].unique().tolist()],'ind':1})

df_mkt_day_hour=pd.merge(df_date,df_hour,on="ind",how="outer")
df_mkt_day_hour=pd.merge(df_mkt_day_hour,df_mkt,on="ind",how="outer")
df_TV_Market=pd.merge(df_TV_Market,df_mkt_day_hour,on=["Date_NYT","Hour_NYT","Market"],how="outer")
df_TV_Market.shape
df_TV_Market['log_count']=df_TV_Market['log_count'].fillna(0).astype(float).astype(int)
del df_TV_Market['ind']
df_TV_Market=df_TV_Market.reset_index()
del df_TV_Market['index']

In [51]:
session_dtype=ga_agg_session['Defined_Media'].unique().tolist()

ga_agg_session['Date_NYT']=ga_agg_session['ga:dateHour'].apply(lambda x: datetime.datetime.strptime(x[:8],"%Y%m%d").date())
ga_agg_session['Hour_NYT']=ga_agg_session['ga:dateHour'].apply(lambda x: int(x[8:]))
ga_agg_session=ga_agg_session[['Date_NYT','Hour_NYT','ga:metro','Defined_Media','ga:sessions']]

In [52]:
holder_ga_date=pd.DataFrame({"Date_NYT":ga_agg_session['Date_NYT'].unique().tolist()})
holder_ga_date['ind']=1

holder_ga_hour=pd.DataFrame({"Hour_NYT":ga_agg_session['Hour_NYT'].unique().tolist()})
holder_ga_hour['ind']=1

holder_ga_DMA=pd.DataFrame({"ga:metro":ga_agg_session['ga:metro'].unique().tolist()})
holder_ga_DMA['ind']=1

holder_ga_Media=pd.DataFrame({"Defined_Media":ga_agg_session['Defined_Media'].unique().tolist()})
holder_ga_Media['ind']=1

df_ga_holder=pd.merge(holder_ga_date,holder_ga_hour,on="ind",how="outer")
df_ga_holder=pd.merge(df_ga_holder,holder_ga_DMA,on="ind",how="outer")
df_ga_holder=pd.merge(df_ga_holder,holder_ga_Media,on="ind",how="outer")

ga_agg_session=pd.merge(ga_agg_session,df_ga_holder,on=['Date_NYT','Hour_NYT','ga:metro','Defined_Media'],how="outer")
del ga_agg_session['ind']
ga_agg_session=ga_agg_session.rename(columns={"ga:metro":"GoogleMarket"})
ga_agg_session['ga:sessions']=ga_agg_session['ga:sessions'].fillna(0).astype(int)

In [53]:
ga_agg_session_all_channels=ga_agg_session.groupby(['Date_NYT','Hour_NYT','GoogleMarket'])['ga:sessions'].sum().to_frame().reset_index()

df_TV_Market=df_TV_Market.sort_values(["Market","Date_NYT","Hour_NYT"])
df_TV_Market['GoogleMarket']=df_TV_Market['Market'].apply(lambda x: Market_TV_to_GoogleAdwords[x])

print(df_TV_Market.shape)
print(df_TV_Market[['GoogleMarket','Date_NYT','Hour_NYT']].drop_duplicates().shape)


(2520, 5)
(2520, 3)


In [54]:
# All sessions accross channels by market
output_df_list_all_channels=[]
writer_DMA_all_channels=pd.ExcelWriter('/home/jian/Projects/Saatva/Analysis/President_Day_2019/output/Saatva_TV_sessions_DMA_all_channels_JL_'+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_7_mkts_agg_all_channels=pd.DataFrame()

GoogleMkt_list=df_TV_Market['GoogleMarket'].unique().tolist()
for mkt,group in df_TV_Market.groupby('Market'):
    df=pd.merge(group,ga_agg_session_all_channels,on=['Date_NYT',"Hour_NYT","GoogleMarket"],how="left")
    df=df.sort_values(['Date_NYT','Hour_NYT']).reset_index()
    del df['index']
    df['sessions_T_0_1']=df['ga:sessions'].tolist()
    df['sessions_T_1_2']=df['ga:sessions'].tolist()[1:]+[0]*1
    df['sessions_T_2_3']=df['ga:sessions'].tolist()[2:]+[0]*2
    df['sessions_T_3_4']=df['ga:sessions'].tolist()[3:]+[0]*3
    df['sessions_T_4_5']=df['ga:sessions'].tolist()[4:]+[0]*4
    df['sessions_T_5_6']=df['ga:sessions'].tolist()[5:]+[0]*5
    
    df['sessions_Cum_1_H']=df['sessions_T_0_1']
    df['sessions_Cum_2_H']=df['sessions_T_0_1']+df['sessions_T_1_2']
    df['sessions_Cum_3_H']=df['sessions_T_0_1']+df['sessions_T_1_2']+df['sessions_T_2_3']
    df['sessions_Cum_4_H']=df['sessions_T_0_1']+df['sessions_T_1_2']+df['sessions_T_2_3']+df['sessions_T_3_4']
    df['sessions_Cum_5_H']=df['sessions_T_0_1']+df['sessions_T_1_2']+df['sessions_T_2_3']+df['sessions_T_3_4']+df['sessions_T_4_5']
    df['sessions_Cum_6_H']=df['sessions_T_0_1']+df['sessions_T_1_2']+df['sessions_T_2_3']+df['sessions_T_3_4']+df['sessions_T_4_5']+df['sessions_T_5_6']
    
    df['Date_NYT']=df['Date_NYT'].astype(str)
    df['Hour_NYT']=df['Hour_NYT'].astype(str)
    df['Hour_NYT']=df['Hour_NYT'].apply(lambda x: x.zfill(2))
    
    df['local_datetime']=df['Date_NYT']+" "+df['Hour_NYT']
    df['local_datetime']=df['local_datetime'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H"))
    df['local_datetime']=df['local_datetime'].apply(lambda x: x-datetime.timedelta(hours=dict_mkt_time_adj[mkt]))
    df=df[df['local_datetime']>=datetime.date(2019,2,11)]
    df=df[df['local_datetime']<datetime.date(2019,2,25)]
    
    df=df[['Market','GoogleMarket','local_datetime','Date_NYT','Hour_NYT','log_count','ga:sessions','sessions_Cum_1_H','sessions_Cum_2_H','sessions_Cum_3_H','sessions_Cum_4_H','sessions_Cum_5_H','sessions_Cum_6_H']]
    
    globals()['output_all_chanels_'+str(mkt.split(" ")[0])]=df
    
    output_df_list_all_channels=output_df_list_all_channels+[df]
    df.to_excel(writer_DMA_all_channels,mkt,index=False)
    output_7_mkts_agg_all_channels=output_7_mkts_agg_all_channels.append(df)

output_7_mkts_agg_all_channels['Market']='Targeted_7_Mkts'
output_7_mkts_agg_all_channels=output_7_mkts_agg_all_channels.groupby(['Date_NYT','Hour_NYT','Market'])['log_count','ga:sessions','sessions_Cum_1_H','sessions_Cum_2_H','sessions_Cum_3_H','sessions_Cum_4_H','sessions_Cum_5_H','sessions_Cum_6_H'].sum().reset_index()
output_7_mkts_agg_all_channels['GoogleMarket']='Targeted_7_Mkts'
output_7_mkts_agg_all_channels['local_datetime']=np.nan
output_7_mkts_agg_all_channels=output_7_mkts_agg_all_channels[df.columns.tolist()]

output_7_mkts_agg_all_channels.to_excel(writer_DMA_all_channels,"Targeted_7_DMAs",index=False)




In [55]:
output_Other_mkts_agg_all_channels=ga_agg_session_all_channels[~ga_agg_session_all_channels['GoogleMarket'].isin(GoogleMkt_list)]
output_Other_mkts_agg_all_channels=output_Other_mkts_agg_all_channels.groupby(['Date_NYT','Hour_NYT'])['ga:sessions'].sum().to_frame().reset_index()
output_Other_mkts_agg_all_channels['sessions_T_0_1']=output_Other_mkts_agg_all_channels['ga:sessions'].tolist()
output_Other_mkts_agg_all_channels['sessions_T_1_2']=output_Other_mkts_agg_all_channels['ga:sessions'].tolist()[1:]+[0]*1
output_Other_mkts_agg_all_channels['sessions_T_2_3']=output_Other_mkts_agg_all_channels['ga:sessions'].tolist()[2:]+[0]*2
output_Other_mkts_agg_all_channels['sessions_T_3_4']=output_Other_mkts_agg_all_channels['ga:sessions'].tolist()[3:]+[0]*3
output_Other_mkts_agg_all_channels['sessions_T_4_5']=output_Other_mkts_agg_all_channels['ga:sessions'].tolist()[4:]+[0]*4
output_Other_mkts_agg_all_channels['sessions_T_5_6']=output_Other_mkts_agg_all_channels['ga:sessions'].tolist()[5:]+[0]*5

output_Other_mkts_agg_all_channels['sessions_Cum_1_H']=output_Other_mkts_agg_all_channels['sessions_T_0_1']
output_Other_mkts_agg_all_channels['sessions_Cum_2_H']=output_Other_mkts_agg_all_channels['sessions_T_0_1']+output_Other_mkts_agg_all_channels['sessions_T_1_2']
output_Other_mkts_agg_all_channels['sessions_Cum_3_H']=output_Other_mkts_agg_all_channels['sessions_T_0_1']+output_Other_mkts_agg_all_channels['sessions_T_1_2']+output_Other_mkts_agg_all_channels['sessions_T_2_3']
output_Other_mkts_agg_all_channels['sessions_Cum_4_H']=output_Other_mkts_agg_all_channels['sessions_T_0_1']+output_Other_mkts_agg_all_channels['sessions_T_1_2']+output_Other_mkts_agg_all_channels['sessions_T_2_3']+output_Other_mkts_agg_all_channels['sessions_T_3_4']
output_Other_mkts_agg_all_channels['sessions_Cum_5_H']=output_Other_mkts_agg_all_channels['sessions_T_0_1']+output_Other_mkts_agg_all_channels['sessions_T_1_2']+output_Other_mkts_agg_all_channels['sessions_T_2_3']+output_Other_mkts_agg_all_channels['sessions_T_3_4']+output_Other_mkts_agg_all_channels['sessions_T_4_5']
output_Other_mkts_agg_all_channels['sessions_Cum_6_H']=output_Other_mkts_agg_all_channels['sessions_T_0_1']+output_Other_mkts_agg_all_channels['sessions_T_1_2']+output_Other_mkts_agg_all_channels['sessions_T_2_3']+output_Other_mkts_agg_all_channels['sessions_T_3_4']+output_Other_mkts_agg_all_channels['sessions_T_4_5']+output_Other_mkts_agg_all_channels['sessions_T_5_6']

output_Other_mkts_agg_all_channels=output_Other_mkts_agg_all_channels[output_Other_mkts_agg_all_channels['Date_NYT']>=datetime.date(2019,2,11)]
output_Other_mkts_agg_all_channels=output_Other_mkts_agg_all_channels[output_Other_mkts_agg_all_channels['Date_NYT']<=datetime.date(2019,2,24)]
output_Other_mkts_agg_all_channels['log_count']=0
output_Other_mkts_agg_all_channels['Market']="Others"

output_Other_mkts_agg_all_channels['GoogleMarket']="Others"
output_Other_mkts_agg_all_channels['local_datetime']=np.nan

output_Other_mkts_agg_all_channels=output_Other_mkts_agg_all_channels[df.columns.tolist()]
output_Other_mkts_agg_all_channels.to_excel(writer_DMA_all_channels,"Other_DMAs",index=False)
writer_DMA_all_channels.save()

In [56]:
for session_type in session_dtype:
    # output_df_list_session_type=[] # If need, do a locals() or globals()
    writer_DMA_session_type=pd.ExcelWriter('/home/jian/Projects/Saatva/Analysis/President_Day_2019/output/Saatva_TV_sessions_DMA_'+session_type+'_JL_'+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
    output_7_mkts_agg_session_type=pd.DataFrame()
    
    ga_agg_session_session_type=ga_agg_session[ga_agg_session['Defined_Media']==session_type]
    
    for mkt,group in df_TV_Market.groupby('Market'):
        df=pd.merge(group,ga_agg_session_session_type,on=['Date_NYT',"Hour_NYT","GoogleMarket"],how="left")
        df=df.sort_values(['Date_NYT','Hour_NYT']).reset_index()
        del df['index']
        df['sessions_T_0_1']=df['ga:sessions'].tolist()
        df['sessions_T_1_2']=df['ga:sessions'].tolist()[1:]+[0]*1
        df['sessions_T_2_3']=df['ga:sessions'].tolist()[2:]+[0]*2
        df['sessions_T_3_4']=df['ga:sessions'].tolist()[3:]+[0]*3
        df['sessions_T_4_5']=df['ga:sessions'].tolist()[4:]+[0]*4
        df['sessions_T_5_6']=df['ga:sessions'].tolist()[5:]+[0]*5

        df['sessions_Cum_1_H']=df['sessions_T_0_1']
        df['sessions_Cum_2_H']=df['sessions_T_0_1']+df['sessions_T_1_2']
        df['sessions_Cum_3_H']=df['sessions_T_0_1']+df['sessions_T_1_2']+df['sessions_T_2_3']
        df['sessions_Cum_4_H']=df['sessions_T_0_1']+df['sessions_T_1_2']+df['sessions_T_2_3']+df['sessions_T_3_4']
        df['sessions_Cum_5_H']=df['sessions_T_0_1']+df['sessions_T_1_2']+df['sessions_T_2_3']+df['sessions_T_3_4']+df['sessions_T_4_5']
        df['sessions_Cum_6_H']=df['sessions_T_0_1']+df['sessions_T_1_2']+df['sessions_T_2_3']+df['sessions_T_3_4']+df['sessions_T_4_5']+df['sessions_T_5_6']

        df['Date_NYT']=df['Date_NYT'].astype(str)
        df['Hour_NYT']=df['Hour_NYT'].astype(str)
        df['Hour_NYT']=df['Hour_NYT'].apply(lambda x: x.zfill(2))

        df['local_datetime']=df['Date_NYT']+" "+df['Hour_NYT']
        df['local_datetime']=df['local_datetime'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H"))
        df['local_datetime']=df['local_datetime'].apply(lambda x: x-datetime.timedelta(hours=dict_mkt_time_adj[mkt]))
        df=df[df['local_datetime']>=datetime.date(2019,2,11)]
        df=df[df['local_datetime']<datetime.date(2019,2,25)]

        df=df[['Market','GoogleMarket','local_datetime','Date_NYT','Hour_NYT','log_count','ga:sessions','sessions_Cum_1_H','sessions_Cum_2_H','sessions_Cum_3_H','sessions_Cum_4_H','sessions_Cum_5_H','sessions_Cum_6_H']]

        globals()['output_all_chanels_'+str(mkt.split(" ")[0])]=df

        # output_df_list_session_type=output_df_list_session_type+[df]
        df.to_excel(writer_DMA_session_type,mkt,index=False)
        output_7_mkts_agg_session_type=output_7_mkts_agg_session_type.append(df)

    output_7_mkts_agg_session_type['GoogleMarket']='Targeted_7_Mkts'
    output_7_mkts_agg_session_type=output_7_mkts_agg_session_type.groupby(['Date_NYT','Hour_NYT','GoogleMarket'])['log_count','ga:sessions','sessions_Cum_1_H','sessions_Cum_2_H','sessions_Cum_3_H','sessions_Cum_4_H','sessions_Cum_5_H','sessions_Cum_6_H'].sum().reset_index()
    output_7_mkts_agg_session_type['Market']="Targeted_7_Mkts"
    output_7_mkts_agg_session_type['local_datetime']=np.nan
    output_7_mkts_agg_session_type=output_7_mkts_agg_session_type[df.columns.tolist()]
    output_7_mkts_agg_session_type.to_excel(writer_DMA_session_type,"Targeted_7_DMAs",index=False)
    
    
    output_Other_mkts_agg_session_type=ga_agg_session_session_type[~ga_agg_session_session_type['GoogleMarket'].isin(GoogleMkt_list)]
    output_Other_mkts_agg_session_type=output_Other_mkts_agg_session_type.groupby(['Date_NYT','Hour_NYT'])['ga:sessions'].sum().to_frame().reset_index()
    output_Other_mkts_agg_session_type['sessions_T_0_1']=output_Other_mkts_agg_session_type['ga:sessions'].tolist()
    output_Other_mkts_agg_session_type['sessions_T_1_2']=output_Other_mkts_agg_session_type['ga:sessions'].tolist()[1:]+[0]*1
    output_Other_mkts_agg_session_type['sessions_T_2_3']=output_Other_mkts_agg_session_type['ga:sessions'].tolist()[2:]+[0]*2
    output_Other_mkts_agg_session_type['sessions_T_3_4']=output_Other_mkts_agg_session_type['ga:sessions'].tolist()[3:]+[0]*3
    output_Other_mkts_agg_session_type['sessions_T_4_5']=output_Other_mkts_agg_session_type['ga:sessions'].tolist()[4:]+[0]*4
    output_Other_mkts_agg_session_type['sessions_T_5_6']=output_Other_mkts_agg_session_type['ga:sessions'].tolist()[5:]+[0]*5

    output_Other_mkts_agg_session_type['sessions_Cum_1_H']=output_Other_mkts_agg_session_type['sessions_T_0_1']
    output_Other_mkts_agg_session_type['sessions_Cum_2_H']=output_Other_mkts_agg_session_type['sessions_T_0_1']+output_Other_mkts_agg_session_type['sessions_T_1_2']
    output_Other_mkts_agg_session_type['sessions_Cum_3_H']=output_Other_mkts_agg_session_type['sessions_T_0_1']+output_Other_mkts_agg_session_type['sessions_T_1_2']+output_Other_mkts_agg_session_type['sessions_T_2_3']
    output_Other_mkts_agg_session_type['sessions_Cum_4_H']=output_Other_mkts_agg_session_type['sessions_T_0_1']+output_Other_mkts_agg_session_type['sessions_T_1_2']+output_Other_mkts_agg_session_type['sessions_T_2_3']+output_Other_mkts_agg_session_type['sessions_T_3_4']
    output_Other_mkts_agg_session_type['sessions_Cum_5_H']=output_Other_mkts_agg_session_type['sessions_T_0_1']+output_Other_mkts_agg_session_type['sessions_T_1_2']+output_Other_mkts_agg_session_type['sessions_T_2_3']+output_Other_mkts_agg_session_type['sessions_T_3_4']+output_Other_mkts_agg_session_type['sessions_T_4_5']
    output_Other_mkts_agg_session_type['sessions_Cum_6_H']=output_Other_mkts_agg_session_type['sessions_T_0_1']+output_Other_mkts_agg_session_type['sessions_T_1_2']+output_Other_mkts_agg_session_type['sessions_T_2_3']+output_Other_mkts_agg_session_type['sessions_T_3_4']+output_Other_mkts_agg_session_type['sessions_T_4_5']+output_Other_mkts_agg_session_type['sessions_T_5_6']

    output_Other_mkts_agg_session_type=output_Other_mkts_agg_session_type[output_Other_mkts_agg_session_type['Date_NYT']>=datetime.date(2019,2,11)]
    output_Other_mkts_agg_session_type=output_Other_mkts_agg_session_type[output_Other_mkts_agg_session_type['Date_NYT']<=datetime.date(2019,2,24)]
    output_Other_mkts_agg_session_type['log_count']=0
    output_Other_mkts_agg_session_type['Market']="Others"
    output_Other_mkts_agg_session_type['GoogleMarket']="Others"
    output_Other_mkts_agg_session_type['local_datetime']=np.nan
    
    output_Other_mkts_agg_session_type=output_Other_mkts_agg_session_type[output_Other_mkts_agg_session_type.columns.tolist()]
    output_Other_mkts_agg_session_type=output_Other_mkts_agg_session_type[df.columns.tolist()]
    output_Other_mkts_agg_session_type.to_excel(writer_DMA_session_type,"Other_DMAs",index=False)
    writer_DMA_all_channels.save()
    

In [57]:
# AdWords by Day


df_TV_Market_by_Day=df_TV_Market.groupby(['Market','GoogleMarket','Date_NYT'])['log_count'].sum().to_frame().reset_index()
df_TV_Market_by_Day


,Market,GoogleMarket,Date_NYT,log_count
0,Charlotte,Charlotte NC,2019-02-11,8
1,Charlotte,Charlotte NC,2019-02-12,0
2,Charlotte,Charlotte NC,2019-02-13,0
...,...,...,...,...
102,Washington DC,Washington DC (Hagerstown MD),2019-02-23,34
103,Washington DC,Washington DC (Hagerstown MD),2019-02-24,56
104,Washington DC,Washington DC (Hagerstown MD),2019-02-25,0


In [58]:
Geo_All_5.head(2)

,Day,Campaign_Type,DMA_name,Impressions,Clicks,Cost
0,2019-02-01,Display,--,177950.0,2026.0,3171.381338
1,2019-02-01,Display,Abilene-Sweetwater TX,1.0,0.0,0.000000


In [95]:
for campaign_type in Geo_All_5['Campaign_Type'].unique().tolist():
    df_adwords=Geo_All_5[Geo_All_5['Campaign_Type']==campaign_type]
    df_adwords=df_adwords.rename(columns={"DMA_name":"GoogleMarket","Day":"Date_NYT"})
    df_adwords['Date_NYT']=df_adwords['Date_NYT'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
    writer_DMA_AdWords=pd.ExcelWriter('/home/jian/Projects/Saatva/Analysis/President_Day_2019/output/Saatva_TV_AdWords_DMA_'+campaign_type+'_JL_'+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
    output_7_mkts_agg_ads_type=pd.DataFrame()
    
    
    for mkt,group in df_TV_Market_by_Day.groupby('Market'):
        df=group[['GoogleMarket','Date_NYT','log_count']]
        df=pd.merge(df,df_adwords,on=['Date_NYT',"GoogleMarket"],how="left")
        df=df.sort_values(['Date_NYT']).reset_index()
        del df['index']
        
        df['CTR']=df['Clicks']/df['Impressions']
        df.to_excel(writer_DMA_AdWords,mkt,index=False)
        df=df[df['Date_NYT']<=datetime.date(2019,2,24)]
        output_7_mkts_agg_ads_type=output_7_mkts_agg_ads_type.append(df)
        
    
    output_Other_mkts_agg_ads_type=df_adwords[~df_adwords['GoogleMarket'].isin(output_7_mkts_agg_ads_type['GoogleMarket'].unique().tolist())]
    
    
    output_7_mkts_agg_ads_type=output_7_mkts_agg_ads_type.groupby(['Date_NYT','Campaign_Type'])['log_count','Impressions',"Clicks","Cost"].sum().reset_index()
    output_7_mkts_agg_ads_type['GoogleMarket']="Targeted_7_Mkts"
    output_7_mkts_agg_ads_type['CTR']=output_7_mkts_agg_ads_type['Clicks']/output_7_mkts_agg_ads_type['Impressions']
    
    output_Other_mkts_agg_ads_type=output_Other_mkts_agg_ads_type.rename(columns={"DMA_name":"GoogleMarket","Day":"Date_NYT"})
    output_Other_mkts_agg_ads_type=output_Other_mkts_agg_ads_type.groupby(["Date_NYT","Campaign_Type"])['Impressions',"Clicks","Cost"].sum().reset_index()
    output_Other_mkts_agg_ads_type['CTR']=output_Other_mkts_agg_ads_type['Clicks']/output_Other_mkts_agg_ads_type['Impressions']
    output_Other_mkts_agg_ads_type['GoogleMarket']="Others"
    output_Other_mkts_agg_ads_type['log_count']=0
    output_Other_mkts_agg_ads_type=output_Other_mkts_agg_ads_type[output_7_mkts_agg_ads_type.columns.tolist()]
    
    output_7_mkts_agg_ads_type=output_7_mkts_agg_ads_type[df.columns.tolist()]
    output_Other_mkts_agg_ads_type=output_Other_mkts_agg_ads_type[df.columns.tolist()]
    
    output_7_mkts_agg_ads_type=output_7_mkts_agg_ads_type[output_7_mkts_agg_ads_type['Date_NYT']<=datetime.date(2019,2,24)]
    output_Other_mkts_agg_ads_type=output_Other_mkts_agg_ads_type[output_Other_mkts_agg_ads_type['Date_NYT']<=datetime.date(2019,2,24)]
    
    
    output_7_mkts_agg_ads_type.to_excel(writer_DMA_AdWords,"Targeted_7_DMAs",index=False)
    output_Other_mkts_agg_ads_type.to_excel(writer_DMA_AdWords,"Other_DMAs",index=False)
    
    
    writer_DMA_AdWords.save()